In [75]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import tensorflow.python.framework.ops as ops
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
tf.compat.v1.disable_eager_execution()

In [37]:
def create_placeholder(n_H0,n_W0,n_C0,n_y):
    X = tf1.placeholder(shape = [None, n_H0,n_W0,n_C0],dtype= tf.float32, name='X')
    y = tf1.placeholder(shape = [None, n_y], dtype= tf.int8, name='y')
    return X,y

In [38]:
def init_params():
    tf1.set_random_seed(1)
    W1 = tf1.get_variable("W1",[4,4,3,8],initializer= tf.initializers.glorot_uniform(seed= 0))
    W2 = tf1.get_variable("W2",[2,2,8,16],initializer= tf.initializers.glorot_uniform(seed= 0))
    params = {"W1":W1,"W2":W2}
    return params

    

In [39]:
def forward_propagation(X,params):
    
    w1 = params.get("W1")
    w2 = params.get("w2")
    s=1
    Z1 = tf.nn.conv2d(X , w1, strides=[1,s,s,1],padding= 'SAME')
    A1 = tf.nn.relu(Z1)
    s= 8
    f= 8
    P1 = tf.nn.max_pool(A1, ksize= [1,f,f,1], strides=[1,s,s,1], padding='SAME')
    
    s=1
    Z2 = tf.nn.conv2d(P1 , w2, strides=[1,s,s,1],padding= 'SAME')
    A2 = tf.nn.relu(Z2)
    s= 4
    f= 4
    P2 = tf.nn.max_pool(A2, ksize= [1,f,f,1], strides=[1,s,s,1], padding='SAME')
    
    F = tf1.layers.flatten(P2)
    num_outputs = 6 
    Z3 = tf1.layers.dense(F,num_outputs= num_outputs)
    
    return Z3
    

In [40]:
def compute_cost(Z3, y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= Z3, labels= y))
    return cost

In [41]:
def random_mini_bathces(x, y, batch_size,seed):
    m = x.shape[0]
    mini_batches = []
    np.random.seed(seed)
    permutation = list(np.random.permutation(m))
    shuffled_x = x[permutation,:]
    shuffled_y = y[permutation,:].reshape((m,y.shape[1]))
    num_complete_minibatches = math.floor(m/batch_size)
    for k in range(0,num_complete_minibatches):
        batch_x = shuffled_x[ k*batch_size:k*batch_size+batch_size,:]
        batch_y = shuffled_y[ k*batch_size:k*batch_size+batch_size,:]
        mini_batches.append((batch_x,batch_y))
    if m % batch_size != 0 :
        batch_x = shuffled_x[ num_complete_minibatches*batch_size:m,:]
        batch_y = shuffled_y[ num_complete_minibatches*batch_size:m,:]
        mini_batches.append((batch_x,batch_y))
    return mini_batches

In [67]:
def model(x, y, x_test, y_test, lr= 0.009, epochs= 100, batch_size= 64, print_cost= True):
    
    ops.reset_default_graph()
    tf1.set_random_seed(1)
    seed = 3
    (m, n_H0, n_W0, n_C0) = x.shape
    n_y = y.shape[1]
    
    costs = []
    
    X , Y = create_placeholder(n_H0, n_W0, n_C0, n_y)
    params = init_params()
    
    Z3 = forward_propagation(X, params)
    
    cost = compute_cost(Z3, Y)
    optimizer = tf1.train.AdamOptimizer(learning_rate=lr).minimize(loss= cost)
    
    init = tf1.global_variables_initializer()
    
    with tf1.Session() as sess:
        sess.run(init)
        for epoch in range(epochs):
            batch_cost = 0 
            num_batches = int(m/batch_size)
            seed +=1
            batches = random_mini_bathces(x, y, batch_size, seed)
            for batch in batches:
                (batch_x,bathc_y) = batch
                
                _,temp_cost = sess.run([optimizer, cost], fedd_dict= {X:batch_x, Y:batch_y})
                batch_cost += temp_cost / num_batches
            
            if print_cost and epoch % 5 == 0:
                print(f"cost after epoch {epoch} : {batch_cost}")
            if print_cost  and epoch % 1 == 0:
                costs.append(batch_cost)
        
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlable('iters (per ten) ')
        plt.title("learning rate = "+ str(lr))
        
        predict_op = tf.argmax(Z3,1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))
        print(accuracy)
        train_acc = accuracy.eval({X:x,Y:y})
        test_acc = accuracy.eval({X:x_test, Y:y_test})
        
        print(f'train accuracy {train_acc}')
        print(f'test accuracy {test_acc}')
        
        return train_acc, test_acc, params
    
    
    
    

In [68]:
from tensorflow.keras.datasets import  mnist

In [69]:
 (x_train, y_train), (x_test, y_test) = mnist.load_data()
    

In [70]:
x_train= x_train.reshape((x_train.shape[0],x_train.shape[1],x_train.shape[2],1))
x_test= x_test.reshape((x_test.shape[0],x_test.shape[1],x_test.shape[2],1))
x_train.shape

(60000, 28, 28, 1)

In [71]:
x_train= x_train/255
x_test = x_test/255

In [72]:
y_train= y_train.reshape((y_train.shape[0],1))
y_test= y_test.reshape((y_test.shape[0],1))

In [73]:
y_train.shape

(60000, 1)

In [77]:
(_,_,params) = model(x_train,y_train,x_test,y_test)

ValueError: Depth of input (1) is not a multiple of input depth of filter (3) for '{{node Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](X, Conv2D/ReadVariableOp)' with input shapes: [?,28,28,1], [4,4,3,8].